# ICS 504 Project

In [1]:
import logging
import os
import pprint

import torch
from torch import nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.optim import SGD
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import yaml

from dataset.semi import SemiDataset
from model.semseg.deeplabv3plus import DeepLabV3Plus
from no_distrib import evaluate
from util.classes import CLASSES
from util.ohem import ProbOhemCrossEntropy2d
from util.utils import count_params, init_log, AverageMeter
from util.dist_helper import setup_distributed

from module_list import *
from build_data import *

In [2]:
dataset='pascal'
method='unimatch_reco'
exp='r101'
split='732'

config = f"configs/{dataset}.yaml"
labeled_id_path = f"splits/{dataset}/{split}/labeled.txt"
unlabeled_id_path = f"splits/{dataset}/{split}/unlabeled.txt"
save_path = f"exp/{dataset}/{method}/{exp}/{split}"
local_rank = 0
port = 1202

In [3]:
cfg = yaml.load(open(config, "r"), Loader=yaml.Loader)
print(cfg)

{'dataset': 'pascal', 'nclass': 21, 'crop_size': 321, 'data_root': 'E:\\ICS_504\\project_datasets\\Pascal\\', 'epochs': 2, 'batch_size': 2, 'lr': 0.001, 'lr_multi': 10.0, 'criterion': {'name': 'CELoss', 'kwargs': {'ignore_index': 255}}, 'conf_thresh': 0.95, 'weak_threshold': 0.7, 'model': 'deeplabv3plus', 'backbone': 'resnet101', 'replace_stride_with_dilation': [False, False, True], 'dilations': [6, 12, 18]}


In [4]:
logger = init_log('global', logging.INFO)
logger.propagate = 0

In [5]:
# rank, world_size = setup_distributed(port=port)
os.makedirs(save_path, exist_ok=True)
writer = SummaryWriter(save_path)
cudnn.enabled = True
cudnn.benchmark = True

In [6]:
model = DeepLabV3Plus(cfg)
optimizer = SGD([{'params': model.backbone.parameters(), 'lr': cfg['lr']},
                {'params': [param for name, param in model.named_parameters() if 'backbone' not in name],
                'lr': cfg['lr'] * cfg['lr_multi']}], lr=cfg['lr'], momentum=0.9, weight_decay=1e-4)


# model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
model.cuda()

# model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[local_rank], broadcast_buffers=False,
#                                                       output_device=local_rank, find_unused_parameters=False)


ema = EMA(model, 0.99)  # Mean teacher model

logger.info('Total params: {:.1f}M\n'.format(count_params(model)))

[2023-05-22 22:45:44,528][    INFO] Total params: 60.2M



In [7]:
if cfg['criterion']['name'] == 'CELoss':
    criterion_l = nn.CrossEntropyLoss(**cfg['criterion']['kwargs']).cuda(local_rank)
elif cfg['criterion']['name'] == 'OHEM':
    criterion_l = ProbOhemCrossEntropy2d(**cfg['criterion']['kwargs']).cuda(local_rank)
else:
    raise NotImplementedError('%s criterion is not implemented' % cfg['criterion']['name'])

criterion_u = nn.CrossEntropyLoss(reduction='none').cuda(local_rank)

In [8]:
trainset_u = SemiDataset(cfg['dataset'], cfg['data_root'], 'train_u',
                            cfg['crop_size'], unlabeled_id_path)
trainset_l = SemiDataset(cfg['dataset'], cfg['data_root'], 'train_l',
                            cfg['crop_size'], labeled_id_path, nsample=len(trainset_u.ids))
valset = SemiDataset(cfg['dataset'], cfg['data_root'], 'val')

trainsampler_l = torch.utils.data.SequentialSampler(trainset_l)
trainloader_l = DataLoader(trainset_l, batch_size=cfg['batch_size'],
                            pin_memory=True, num_workers=1, drop_last=True, sampler=trainsampler_l)
trainsampler_u = torch.utils.data.SequentialSampler(trainset_u)
trainloader_u = DataLoader(trainset_u, batch_size=cfg['batch_size'],
                            pin_memory=True, num_workers=1, drop_last=True, sampler=trainsampler_u)
valsampler = torch.utils.data.SequentialSampler(valset)
valloader = DataLoader(valset, batch_size=1, pin_memory=True, num_workers=1,
                        drop_last=False, sampler=valsampler)


In [9]:
total_iters = len(trainloader_u) * cfg['epochs']
previous_best = 0.0
epoch = -1

logger.info(f"Total iters: {total_iters}")

[2023-05-22 22:45:44,556][    INFO] Total iters: 9850


In [10]:
if os.path.exists(os.path.join(save_path, 'latest.pth')):
    checkpoint = torch.load(os.path.join(save_path, 'latest.pth'))
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch = checkpoint['epoch']
    previous_best = checkpoint['previous_best']
    
    logger.info('************ Load from checkpoint at epoch %i\n' % epoch)

In [11]:
for epoch in range(epoch + 1, cfg['epochs']):
    logger.info('===========> Epoch: {:}, LR: {:.5f}, Previous best: {:.2f}'.format(
        epoch, optimizer.param_groups[0]['lr'], previous_best))

    total_loss = AverageMeter()
    total_loss_x = AverageMeter()
    total_loss_s = AverageMeter()
    total_loss_w_fp = AverageMeter()
    total_mask_ratio = AverageMeter()

    # trainloader_l.sampler.set_epoch(epoch)
    # trainloader_u.sampler.set_epoch(epoch)

    loader = zip(trainloader_l, trainloader_u, trainloader_u)

    for i, ((img_x, mask_x),
            (img_u_w, img_u_s1, img_u_s2, ignore_mask, cutmix_box1, cutmix_box2),
            (img_u_w_mix, img_u_s1_mix, img_u_s2_mix, ignore_mask_mix, _, _)) in enumerate(loader):
        
        img_x, mask_x = img_x.cuda(), mask_x.cuda()
        img_u_w = img_u_w.cuda()
        img_u_s1, img_u_s2, ignore_mask = img_u_s1.cuda(), img_u_s2.cuda(), ignore_mask.cuda()
        cutmix_box1, cutmix_box2 = cutmix_box1.cuda(), cutmix_box2.cuda()
        img_u_w_mix = img_u_w_mix.cuda()
        img_u_s1_mix, img_u_s2_mix = img_u_s1_mix.cuda(), img_u_s2_mix.cuda()
        ignore_mask_mix = ignore_mask_mix.cuda()

        with torch.no_grad():
            model.eval()

            pred_u_w_mix = model(img_u_w_mix).detach()
            conf_u_w_mix = pred_u_w_mix.softmax(dim=1).max(dim=1)[0]
            mask_u_w_mix = pred_u_w_mix.argmax(dim=1)

        img_u_s1[cutmix_box1.unsqueeze(1).expand(img_u_s1.shape) == 1] = \
            img_u_s1_mix[cutmix_box1.unsqueeze(1).expand(img_u_s1.shape) == 1]
        img_u_s2[cutmix_box2.unsqueeze(1).expand(img_u_s2.shape) == 1] = \
            img_u_s2_mix[cutmix_box2.unsqueeze(1).expand(img_u_s2.shape) == 1]

        model.train()
        ema.model.train()

        with torch.no_grad():
            train_u_data = torch.cat((img_u_w, img_u_s1, img_u_s2))            
            pred_u = ema.model(train_u_data)            
            pred_u_large_raw = F.interpolate(pred_u, size=ignore_mask.shape[1:], mode="bilinear", align_corners=True)
            pseudo_logits, pseudo_labels = torch.max(torch.softmax(pred_u_large_raw, dim=1), dim=1)

             # random scale images first
            train_u_aug_data, train_u_aug_label, train_u_aug_logits = \
                batch_transform(train_u_data, pseudo_labels, pseudo_logits,
                                [321, 321], (0.5, 1.5), apply_augmentation=False)

            # apply mixing strategy: cutout, cutmix or classmix
            train_u_aug_data, train_u_aug_label, train_u_aug_logits = \
                generate_unsup_data(train_u_aug_data, train_u_aug_label, train_u_aug_logits, mode="cutout")

            # apply augmentation: color jitter + flip + gaussian blur
            train_u_aug_data, train_u_aug_label, train_u_aug_logits = \
                batch_transform(train_u_aug_data, train_u_aug_label, train_u_aug_logits,
                                [321, 321], (1.0, 1.0), apply_augmentation=True)

        num_lb, num_ulb = img_x.shape[0], img_u_w.shape[0]

        preds, preds_fp, rep_xu = model(torch.cat((img_x, img_u_w)), need_fp=True, need_rep=True)
        pred_x, pred_u_w = preds.split([num_lb, num_ulb])
        pred_u_w_fp = preds_fp[num_lb:]

        pred_u_s1s2, rep_s1s2 = model(torch.cat((img_u_s1, img_u_s2)), need_rep=True)
        pred_u_s1, pred_u_s2 = pred_u_s1s2.chunk(2)

        pred_u_w = pred_u_w.detach()
        conf_u_w = pred_u_w.softmax(dim=1).max(dim=1)[0]
        mask_u_w = pred_u_w.argmax(dim=1)

        mask_u_w_cutmixed1, conf_u_w_cutmixed1, ignore_mask_cutmixed1 = \
            mask_u_w.clone(), conf_u_w.clone(), ignore_mask.clone()
        mask_u_w_cutmixed2, conf_u_w_cutmixed2, ignore_mask_cutmixed2 = \
            mask_u_w.clone(), conf_u_w.clone(), ignore_mask.clone()

        mask_u_w_cutmixed1[cutmix_box1 == 1] = mask_u_w_mix[cutmix_box1 == 1]
        conf_u_w_cutmixed1[cutmix_box1 == 1] = conf_u_w_mix[cutmix_box1 == 1]
        ignore_mask_cutmixed1[cutmix_box1 == 1] = ignore_mask_mix[cutmix_box1 == 1]

        mask_u_w_cutmixed2[cutmix_box2 == 1] = mask_u_w_mix[cutmix_box2 == 1]
        conf_u_w_cutmixed2[cutmix_box2 == 1] = conf_u_w_mix[cutmix_box2 == 1]
        ignore_mask_cutmixed2[cutmix_box2 == 1] = ignore_mask_mix[cutmix_box2 == 1]

        loss_x = criterion_l(pred_x, mask_x)

        loss_u_s1 = criterion_u(pred_u_s1, mask_u_w_cutmixed1)
        loss_u_s1 = loss_u_s1 * ((conf_u_w_cutmixed1 >= cfg['conf_thresh']) & (ignore_mask_cutmixed1 != 255))
        loss_u_s1 = loss_u_s1.sum() / (ignore_mask_cutmixed1 != 255).sum().item()

        loss_u_s2 = criterion_u(pred_u_s2, mask_u_w_cutmixed2)
        loss_u_s2 = loss_u_s2 * ((conf_u_w_cutmixed2 >= cfg['conf_thresh']) & (ignore_mask_cutmixed2 != 255))
        loss_u_s2 = loss_u_s2.sum() / (ignore_mask_cutmixed2 != 255).sum().item()

        loss_u_w_fp = criterion_u(pred_u_w_fp, mask_u_w)
        loss_u_w_fp = loss_u_w_fp * ((conf_u_w >= cfg['conf_thresh']) & (ignore_mask != 255))
        loss_u_w_fp = loss_u_w_fp.sum() / (ignore_mask != 255).sum().item()


        with torch.no_grad():
            pred_all = torch.cat((preds, pred_u_s1, pred_u_s2))
            train_u_aug_mask = train_u_aug_logits.ge(cfg['weak_threshold']).float()
            mask_all = torch.cat(((mask_x.unsqueeze(1) >= 0).float(), train_u_aug_mask.unsqueeze(1)))
            mask_all = F.interpolate(mask_all, size=pred_all.shape[2:], mode='nearest')

            label_l = F.interpolate(F.one_hot(mask_x).to(torch.float), size=pred_all.shape[2:], mode='nearest')
            train_u_aug_label[train_u_aug_label < 0] = 0
            label_u = F.interpolate(F.one_hot(train_u_aug_label).to(torch.float), size=pred_all.shape[2:], mode='nearest')
            label_all = torch.cat((label_l, label_u))

            prob_l = torch.softmax(pred_x, dim=1)
            prob_u = torch.softmax(pred_u, dim=1)
            prob_all = torch.cat((prob_l, prob_u))

            rep_all = torch.cat((rep_xu, rep_s1s2))

            reco_loss = compute_reco_loss(rep_all, label_all, mask_all, prob_all, cfg['conf_thresh'],
                                          0.5, 256, 512)



        loss = (loss_x + loss_u_s1 * 0.25 + loss_u_s2 * 0.25 + loss_u_w_fp * 0.5) / 2.0

        # torch.distributed.barrier()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss.update(loss.item())
        total_loss_x.update(loss_x.item())
        total_loss_s.update((loss_u_s1.item() + loss_u_s2.item()) / 2.0)
        total_loss_w_fp.update(loss_u_w_fp.item())
        
        mask_ratio = ((conf_u_w >= cfg['conf_thresh']) & (ignore_mask != 255)).sum().item() / \
            (ignore_mask != 255).sum()
        total_mask_ratio.update(mask_ratio.item())

        iters = epoch * len(trainloader_u) + i
        lr = cfg['lr'] * (1 - iters / total_iters) ** 0.9
        optimizer.param_groups[0]["lr"] = lr
        optimizer.param_groups[1]["lr"] = lr * cfg['lr_multi']
        
        writer.add_scalar('train/loss_all', loss.item(), iters)
        writer.add_scalar('train/loss_x', loss_x.item(), iters)
        writer.add_scalar('train/loss_s', (loss_u_s1.item() + loss_u_s2.item()) / 2.0, iters)
        writer.add_scalar('train/loss_w_fp', loss_u_w_fp.item(), iters)
        writer.add_scalar('train/mask_ratio', mask_ratio, iters)
    
        if (i % (len(trainloader_u) // 8) == 0):
            logger.info('Iters: {:}, Total loss: {:.3f}, Loss x: {:.3f}, Loss s: {:.3f}, Loss w_fp: {:.3f}, Mask ratio: '
                        '{:.3f}'.format(i, total_loss.avg, total_loss_x.avg, total_loss_s.avg,
                                        total_loss_w_fp.avg, total_mask_ratio.avg))

    eval_mode = 'sliding_window' if cfg['dataset'] == 'cityscapes' else 'original'
    mIoU, iou_class = evaluate(model, valloader, eval_mode, cfg)

    for (cls_idx, iou) in enumerate(iou_class):
        logger.info('***** Evaluation ***** >>>> Class [{:} {:}] '
                    'IoU: {:.2f}'.format(cls_idx, CLASSES[cfg['dataset']][cls_idx], iou))
    logger.info('***** Evaluation {} ***** >>>> MeanIoU: {:.2f}\n'.format(eval_mode, mIoU))
    
    writer.add_scalar('eval/mIoU', mIoU, epoch)
    for i, iou in enumerate(iou_class):
        writer.add_scalar('eval/%s_IoU' % (CLASSES[cfg['dataset']][i]), iou, epoch)

    is_best = mIoU > previous_best
    previous_best = max(mIoU, previous_best)
    
    checkpoint = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch,
        'previous_best': previous_best,
    }
    torch.save(checkpoint, os.path.join(save_path, 'latest.pth'))
    if is_best:
        torch.save(checkpoint, os.path.join(save_path, 'best.pth'))

[2023-05-22 22:45:44,644][    INFO] ===========> Epoch: 0, LR: 0.00100, Previous best: 0.00
E:\ICS_504\UniMatch\venv\lib\site-packages\torchvision\transforms\functional.py:417: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


IndexError: The shape of the mask [8, 321, 321] at index 0 does not match the shape of the indexed tensor [12, 81, 81, 256] at index 0